## Populate Aitraffic RDF database

This notebook reports the main steps to download CSV files, process them and create our airtraffic RDF dataset from them accordingly to our ontology. 

To measure execution time in Jupyter notebooks: <code>pip install ipython-autotime</code>

We need to install <code>RDFLib</code>

<code>pip3 install rdflib </code> [Documentation](https://rdflib.readthedocs.io/en/stable/gettingstarted.html)

In [ ]:
# required libraries
import pandas as pd
import os
from pathlib import Path
import gc
from datetime import datetime

In [ ]:
# parameters and URLs
path = str(Path(os.path.abspath(os.getcwd())).absolute())

flightsPath = path + '/airtrafficDB/data/flights/'
airportsURL = path + '/airtrafficDB/data/airport-codes.csv'
aircraftURL = path + '/airtrafficDB/data/aircraft.csv'
aircraftTypesURL = path + '/airtrafficDB/data/doc8643AircraftTypes.csv'
airlineURL = path + '/airtrafficDB/data/airline.csv'
manufacturersURL = path + '/airtrafficDB/data/doc8643Manufacturers.csv'

# country codes
countriesURL = path + '/airtrafficDB/data/wikipedia-iso-country-codes.csv'

#cities codes
citiesURL = path + '/airtrafficDB/data/world-cities_csv.csv'

# saving folder
savePath =  path + '/airtrafficDB/rdf/'

In [ ]:
# Load the required libraries
from rdflib import Graph, Literal, RDF, URIRef, Namespace
# rdflib knows about some namespaces, like XSD
from rdflib.namespace import XSD

# Construct the country and the aitraffic ontology namespaces not known by RDFlib
CNS = Namespace("http://eulersharp.sourceforge.net/2003/03swap/countries#")
ATO = Namespace("http://www.unipd.albertopiva.it/db2/airtrafficOntology#")

## Flights
In this section we model the flights. Since the data we need for the aircrafts are stored in the same file as the flights we also model each aircraft and store them in a separate turtle file.

In [ ]:
#create the graph for the flights
g = Graph()

#create the graph for the aircrafts
h = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("ato", ATO)

h.bind("xsd", XSD)
h.bind("countries", CNS)
h.bind("ato", ATO)

In [ ]:
%%time 
#measure execution time
#some counters for diagnostics
count = 0 
prev = 0
unkCount = 1 # counter for unkown aircraft
cFiles = 1 # counter for output files

# We have multiple files for the flights
for filename in os.listdir(flightsPath):
    
    filePath = str(flightsPath) + str(filename)
    flights = pd.read_csv(filePath, sep=',', index_col="ECTRL ID")
    
    print("*** START READING FROM FILE " + str(filename) + " ***")
    
    #iterate over the flights dataframe
    for index, row in flights.iterrows():
        
        # Create the node to add to the Graph
        Flight = URIRef(ATO[str(index)])
        count+=1
        
        # Add triples using store's add() method.
        g.add((Flight, RDF.type, ATO['Flight']))
        # Put timestamps in the right dateTime format
        fob = str(row['FILED OFF BLOCK TIME'])
        filedOffBlock = datetime.strptime(fob,'%d-%m-%Y %H:%M:%S',).strftime('%Y-%m-%dT%H:%M:%S')
        g.add((Flight, ATO['filedOffBlock'], Literal(filedOffBlock, datatype=XSD.dateTime)))
        far = str(row['FILED ARRIVAL TIME'])
        filedArrival = datetime.strptime(far,'%d-%m-%Y %H:%M:%S',).strftime('%Y-%m-%dT%H:%M:%S')
        g.add((Flight, ATO['filedArrival'], Literal(filedArrival, datatype=XSD.dateTime)))
        aob = str(row['ACTUAL OFF BLOCK TIME'])
        actualOffBlock = datetime.strptime(aob,'%d-%m-%Y %H:%M:%S',).strftime('%Y-%m-%dT%H:%M:%S')
        g.add((Flight, ATO['actualOffBlock'], Literal(actualOffBlock, datatype=XSD.dateTime)))
        arr = str(row['ACTUAL ARRIVAL TIME'])
        actualArrival = datetime.strptime(arr,'%d-%m-%Y %H:%M:%S',).strftime('%Y-%m-%dT%H:%M:%S')
        g.add((Flight, ATO['actualArrival'], Literal(actualArrival, datatype=XSD.dateTime)))
        
        # Add the remaining properties
        g.add((Flight, ATO['marketSegment'], Literal(row['STATFOR Market Segment'], datatype=XSD.string)))
        g.add((Flight, ATO['distance'], Literal(row['Actual Distance Flown (nm)'], datatype=XSD.integer))) 

        ## handle airports, discard unknown airports
        if(not (str(row['ADEP'])=='ZZZZ')):
            Adep = URIRef(ATO[row['ADEP']])
            g.add((Flight, ATO['departs'], Adep))  
        if(not (str(row['ADES'])=='ZZZZ')):
            Ades = URIRef(ATO[row['ADES']])
            g.add((Flight, ATO['arrives'], Ades))
        
        ## handle airline, discard unknown operator
        if(not (str(row['AC Operator'])=='ZZZ')):
            Airline = URIRef(ATO[row['AC Operator']])
            #add the edge connecting the Flight and the Airline performing it
            g.add((Flight, ATO['performedBy'], Airline))
            
        ## Handle aircraft, discard unknwon aircrafts with unknown model 
        # Check if we have aircraft data
        if(not pd.isnull(row['AC Registration'])): 
            airCode = str(row['AC Registration'])    
        # Otherwise we create an unknown aircraft so we can store data about aircraft model    
        elif(not (str(row['AC Type'])=='ZZZZ')):  
            airCode = "UNK" + str(unkCount)
            unkCount += 1
            
        # Create the node to add to the graph   
        Aircraft = URIRef(ATO[airCode])
        h.add((Aircraft, RDF.type, ATO['Aircraft']))
        # Handle aircraft models, discard unknown models
        if(not (str(row['AC Type'])=='ZZZZ')):
            AircraftModel = URIRef(ATO[row['AC Type']])
            #add the edge connecting the Aircraft and its Aircraft model
            h.add((Aircraft, ATO['hasType'], AircraftModel))
        #add the edge connecting the Flight and the Aircraft performing it
        h.add((Aircraft, ATO['performs'], Flight))

        ##Diagnostics to keep track of progress
        if(count%150000 == 0): print("added 150K flights")
        if(count%350000 == 0):
            
            # we save the output in a ttl file 
            ttlFname = 'flights' + str(cFiles) + '.ttl'
            ttlAname = 'aircrafts' + str(cFiles) + '.ttl'
            cFiles+=1
            
            print("--- saving flights serialization in " + ttlFname + " ---")
            with open(savePath + ttlFname, 'w') as file:
                file.write(g.serialize(format='turtle'))
            print("--- SAVED ------------------------------------------------")
            
            print("--- saving aircrafts serialization in " + ttlAname + " ---")
            with open(savePath + ttlAname, 'w') as file:
                file.write(h.serialize(format='turtle'))
            print("--- SAVED ------------------------------------------------")      
            # we free some memory
            del g
            del h
            gc.collect()
            # we initialize the graphs again to manage memory load
            
            g = Graph()
            g.bind("xsd", XSD)
            g.bind("countries", CNS)
            g.bind("ato", ATO)
            
            h = Graph()
            h.bind("xsd", XSD)
            h.bind("countries", CNS)
            h.bind("ato", ATO)
            
            
    print('*** END-OF-FILE: ADDED ' + str(count-prev) + ' FLIGHTS FROM FILE ' + str(filename) + ' ***')        
    prev = count
        
print('***')        
print('*** ADDED IN TOTAL ' + str(count) + ' FLIGHTS ***')
print('***')

In [ ]:
%%time
# print the remaining data for the flights in the Turtle format
ttlname = 'flights' + str(cFiles) + '.ttl'
print("--- saving serialization for the last flights ---")
with open(savePath + ttlname, 'w') as file:
    file.write(g.serialize(format='turtle'))

In [ ]:
%%time
# print the remaining data for the aircrafts in the Turtle format
ttlname = 'aircrafts' + str(cFiles) + '.ttl'
print("--- saving serialization for the last aircrafts ---")
with open(savePath + ttlname, 'w') as file:
    file.write(h.serialize(format='turtle'))

## Airports and Cities
In this section we model the airports and the cities. In order to match efficiently each airport with the correct city we create a dictionary of cities indexed with the iso-code of the country each city belongs to. Then, if we find data about new cities in the airports file we add them to our cities modelling. 

### Indexing the Cities
Firstly, we create the dictionary with the cities indexed by country. 

In [ ]:
# Load the CSV files in memory
cities = pd.read_csv(citiesURL, sep=',', index_col='geonameid', keep_default_na=False, na_values=['_'])
cities.info()
countries = pd.read_csv(countriesURL, sep=';', index_col='English short name lower case', keep_default_na=False, na_values=['_'])
countries.info()

In [ ]:
# Creation of the structure
citiesByCountry = {}
    
#iterate over the cities dataframe
for index, row in cities.iterrows():
    
    ## Match country
    cName = str(row['country'])
    # check if the country exists
    # country.index == x returns an array of booleans, thus we need to use the any() method
    if((countries.index == cName).any() == True):
        #get the country code, convert to string and get the lower case to match the country codes in the contology 
        code = str(countries[countries.index == cName]['Alpha-2 code'][0]).lower()
        if(code not in citiesByCountry): #create a new country before appending the city
            citiesByCountry.update({code:[]})
        #append the city to the country   
        citiesByCountry[code].append([row['name'], "C"+str(index)])
        
print("Dictionary Created")

### Airports Modelling

In [ ]:
# Load the CSV files in memory
airports = pd.read_csv(airportsURL, sep=',', index_col='ident')
airports.info()

In [ ]:
#create a new graph for the airports
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("ato", ATO)

In [ ]:
%%time
#measure execution time
count=0 #counter for diagnostic purpose
noMatch=0
newCities=[]

#iterate over the airports dataframe
for index, row in airports.iterrows():
    
    # Create the node to add to the Graph
    Airport = URIRef(ATO[str(index)])
    g.add((Airport, RDF.type, ATO['Airport']))
    count+=1
    
    # Add triples using store's add() method.
    g.add((Airport, ATO['airportName'], Literal(row['name'], datatype=XSD.string)))
    g.add((Airport, ATO['airportType'], Literal(row['type'], datatype=XSD.string)))

    ## Handle cities, note that not all airports have data about city
    # but every airport that have city data also have iso_country data
    if(not pd.isnull(row['municipality'])):
        
        #strip space and uppercase to avoid mismatching for multiple names cities
        cName = str(row['municipality']).replace(" ","").lower()
        
        matched = False
        try:
            for city in citiesByCountry[str(row['iso_country']).lower()]:
                
                check = city[0].replace(" ","").lower() 
                
                if(cName == check):
                    
                    #create the RDF node
                    City = URIRef(ATO[str(city[1])])
                    #add the edge connecting the Airport and the City
                    g.add((Airport, ATO['located'], City))
                    matched = True
                    # once we find the correct city there is no need to keep iterating
                    break 
            # no match with existing cities, we create a new city        
            if(not matched): 
                noMatch += 1
                newCode = "N"+str(noMatch).zfill(6)  
                newCities.append([newCode, str(row['municipality']), str(row['iso_country']).lower()])
                #create the RDF node
                City = URIRef(ATO[newCode])
                #add the edge connecting the Airport and the City
                g.add((Airport, ATO['located'], City))
                
                
        # We skip airport countries not in the dictionary
        except KeyError: continue
            
    if(count%10000==0): print("*** 10K Airports added ***")
               
print('***')        
print('*** ADDED IN TOTAL ' + str(count) + ' AIRPORTS ***')
print('***')    

In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'airports.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

### Cities Modelling
We model the cities using the dictionary we already created so we avoid matching again the countries and cities files.

In [ ]:
#create a new graph for the cities
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("ato", ATO)

In [ ]:
%%time 
#measure execution time
oldCount=0
newCount=0
#iterate over the dictionary
for country in citiesByCountry:
    for city in citiesByCountry[country]:
        # Create the node to add to the Graph
        City = URIRef(ATO[city[1]])
        g.add((City, RDF.type, ATO['City']))
        # Add triples using store's add() method.
        g.add((City, ATO['cityName'], Literal(city[0], datatype=XSD.string)))
        # create the RDF node
        Country = URIRef(CNS[country])
        # add the edge connecting the City and the Country 
        g.add((City, ATO['belongsTo'], Country))
        oldCount+=1

print("Modelled " + str(oldCount) + " cities from Dictionary")        

#iterate over the list of new cities founded when modelling airports
for city in newCities:
    # Create the node to add to the Graph
        City = URIRef(ATO[city[0]])
        g.add((City, RDF.type, ATO['City']))
        # Add triples using store's add() method.
        g.add((City, ATO['cityName'], Literal(city[1], datatype=XSD.string)))
        # create the RDF node
        Country = URIRef(CNS[city[2]])
        # add the edge connecting the City and the Country 
        g.add((City, ATO['belongsTo'], Country))
        newCount+=1

print("Modelled " + str(newCount) + " cities from AIRPORTS")  

In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'cities.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

## AircraftModel
In this section we model the Aircraft Models. We have to consider two file:
- the first one gives me the name and the numberOfSeats;
- the second one gives me information about engine and manufacturer code

In [ ]:
# Load the CSV files in memory
ac_model = pd.read_csv(aircraftURL, sep=';',header=None, index_col=1).dropna()
ac_model.info()

In [ ]:
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("ato", ATO)

In [ ]:
%%time 
#measure execution time

#iterate over the aircraft-model dataframe
for index, row in ac_model.iterrows():
    if index == "\\N" or index == "N/A" :
        continue
    # Create the node to add to the Graph
    Aircraft = URIRef(ATO[index])
    
    # Add triples using store's add() method.
    g.add((Aircraft, RDF.type, ATO['AircraftModel']))   
    g.add((Aircraft, ATO['modelName'], Literal(row[0], datatype=XSD.string)))
    if not (row[3] == "\\N" or row[3] == "N/A" ):
        g.add((Aircraft, ATO['numberOfSeats'], Literal(row[3], datatype=XSD.integer)))


In [ ]:
# Load the CSV files in memory
ac_type = pd.read_csv(aircraftTypesURL, sep=',', index_col='Designator').dropna()
ac_type.info()

In [ ]:
%%time 
#measure execution time

#iterate over the aircraft-model dataframe
for index, row in ac_type.iterrows():
    if index == "\\N" or index == "N/A" :
        continue
    # Create the node to add to the Graph
    Aircraft = URIRef(ATO[index])
    
    #Add triples about engine information
    g.add((Aircraft, ATO['engineType'], Literal(row['EngineType'], datatype=XSD.string)))
    if(row['EngineCount'] != "C"): #Add triple only if EngineCount is a number
        g.add((Aircraft, ATO['engineCount'], Literal(row['EngineCount'], datatype=XSD.integer)))
    
    
    #find the manufacturer
    man = row['ManufacturerCode'].replace(" ","").replace("(","").replace(")","")
    Manufacturer = URIRef(ATO[man])
    g.add((Aircraft, ATO['builtBy'], Manufacturer))


In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'aircraft-models.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))


## Manufacturers 
In this section we model the Manufacturers

In [ ]:
# Load the CSV files in memory
manufacturers = pd.read_csv(manufacturersURL, sep=',', index_col='Code').dropna()
manufacturers.info()

In [ ]:
#load the country codes
# we need to convert NaN values to something else otherwise NA strings are converted to NaN -> problem with Namibia
countries = pd.read_csv(countriesURL, sep=';', index_col='English short name lower case', keep_default_na=False, na_values=['_'])

In [ ]:
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("ato", ATO)

In [ ]:
%%time
#measure execution time

#iterate over the manifacturers dataframe
for index, row in manufacturers.iterrows():
    
    if index == "(any manufacturer)" or row['Name'].startswith("see"):
        continue
        
    # Create the node to add to the Graph
    ind = index.replace(" ","").replace("(","").replace(")","")
    Manufacturer = URIRef(ATO[ind])
    
    # Add triples using store's add() method.
    g.add((Manufacturer, RDF.type, ATO.Manufacturer))

    #Handle the name of the manufacturer 
    splits = row['Name'].split("(")
    name = splits[0].strip()
    #add the code and the full name
    g.add((Manufacturer, ATO['manufacturerName'], Literal(name, datatype=XSD.string)))
    
    # Handle country, find the alpha-2 code of the country
    country = splits[1].replace(")","").strip()
    for ind,cntr in countries.iterrows():
        if ind.lower() == country.lower():
            Country = URIRef(CNS[cntr['Alpha-2 code']])
            g.add((Manufacturer, ATO['hasNationality'], Country))
    

In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'manufacturers.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

## Airline Companies
In this section we model the Airline Companies. We need to consider an important aspect for reading the file:
- the csv file about airline does not have the header

In [ ]:
# Load the CSV files in memory
airline = pd.read_csv(airlineURL, sep=',', index_col='ICAO').dropna()
airline.info()

In [ ]:
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("ato", ATO)

In [ ]:
%%time 
#measure execution time

lista = []
dupl=[]
#iterate over the airline dataframe
for index, row in airline.iterrows():
    if index == "nan" or str(index).strip() == "":
        continue
    if index not in lista:
        lista.append(index)
        # Create the node to add to the Graph
        Airline = URIRef(ATO[str(index).strip()])
        # Add triples using store's add() method.
        g.add((Airline, RDF.type, ATO.Airline))
        # Add the code and the full name
        g.add((Airline, ATO['airlineName'], Literal(row['Airline'], datatype=XSD.string)))

        # Handle countries, find the alpha-2 code of the country
        country = row['Country/Region']
        for ind,cntr in countries.iterrows():
            if ind.lower() == country.lower():
                Country = URIRef(CNS[cntr['Alpha-2 code']])
                g.add((Airline, ATO['hasNationality'], Country))

In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'airlines.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))
